In [ ]:
# PC Info
# CPU: Intel(R) Core(TM) i7-3630QM CPU @ 2.40GHz
# RAM: 8GB
# Disk: HDD

In [ ]:
from django.db.models import Count
from django_prepared_query import BindParam
from books.models import Publisher, Book, Author

In [2]:
prepared_qs = Publisher.prepared_objects.filter(id=BindParam('id')).prepare()
qs = Publisher.objects.all()
%timeit -n 1000 prepared_qs.execute(id=160)
%timeit -n 1000 list(qs.filter(id=160))

257 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
628 µs ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
prepared_qs = Book.prepared_objects.filter(authors__age=BindParam('author_age'), pages__lte=BindParam('max_pages'), 
                                           pages__gte=BindParam('min_pages')).prepare()
qs = Book.objects.all()
%timeit -n 500 prepared_qs.execute(author_age=35, min_pages=100, max_pages=1000)
%timeit -n 500 list(qs.filter(authors__age=35, pages__gte=100, pages__lte=1000))

579 µs ± 261 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
1.76 ms ± 79.2 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [4]:
prepared_qs = Book.prepared_objects.select_related('publisher').filter(id=BindParam('id')).prepare()
qs = Book.objects.select_related('publisher')
%timeit -n 1000 prepared_qs.execute(id=160)
%timeit -n 1000 list(qs.filter(id=160))

503 µs ± 144 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.57 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
prepared_qs = Book.prepared_objects.annotate(num_of_authors=Count('authors')).filter(id=160).prepare()
qs = Book.objects.annotate(num_of_authors=Count('authors')).filter(id=160)
%timeit -n 1000 prepared_qs.execute()
%timeit -n 1000 list(qs.all())

444 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.98 ms ± 552 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
prepared_qs = Book.prepared_objects.prefetch_related('authors').filter(id__in=[160, 161]).prepare()
qs = Book.objects.prefetch_related('authors').filter(id__in=[160, 161])
%timeit -n 500 prepared_qs.execute()
%timeit -n 500 list(qs.all())

610 µs ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
4.01 ms ± 617 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [11]:
prepared_qs = Book.prepared_objects.all()[0:5].prepare()
qs = Book.objects.all()[0:5]
%timeit -n 1000 prepared_qs.execute()
%timeit -n 1000 list(qs.all())

465 µs ± 87.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
725 µs ± 135 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
from django.db.models import Count
prepared_qs = Book.prepared_objects.annotate(num_of_authors=Count('authors')). \
    filter(num_of_authors__gte=BindParam('num_of_authors')).prepare()
qs = Book.objects.annotate(num_of_authors=Count('authors'))
%timeit -n 500 prepared_qs.execute(num_of_authors=6)
%timeit -n 500 list(qs.filter(num_of_authors__gte=6))

1.48 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
2.42 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
